# Preprocessing the Phoneme Sequences
Here I have to process the phoneme sequences so that they can be fed into a Keras embedding layer

In [1]:
#preprocessing for using whole-sequence embedding approach
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import csv
import functools
import pandas as pd
import random

random.seed(285)

# need to read in csv file with pairs and labels
train = pd.read_csv("capstone_train_and_test/new_train.csv")
vocabulary_size = 39 #aka number of different phonemes
max_len = 6 #maximum size of a phoneme sequence. Average sequence is 6.34 phonemes
tokenizer = Tokenizer(num_words = vocabulary_size)
tokenizer.fit_on_texts(train["phonemic_transcriptions_1"]) #finds number of tokens (phonemes in this case)
train_sequences_1 = tokenizer.texts_to_sequences(train["phonemic_transcriptions_1"]) #translates all words to lists of integers
train_sequences_2 = tokenizer.texts_to_sequences(train["phonemic_transcriptions_2"])
train_data_1_concrete = pad_sequences(train_sequences_1, maxlen = max_len, padding = "post")
train_data_2_concrete = pad_sequences(train_sequences_2, maxlen = max_len, padding = "post")
print(train_data_1_concrete[10])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


[ 5  6  7 20  6  0]


In [2]:
raw_labels = train["rhyme_percentile"]*10
categorized_labels_concrete = to_categorical(raw_labels) # the `y` label we're trying to fit to
train_data_1_concrete = list(train_data_1_concrete)
train_data_2_concrete = list(train_data_2_concrete)
categorized_labels_concrete = list(categorized_labels_concrete)

In [3]:
#find info about lists


In [ ]:
train_data_1 = train_data_1_concrete
train_data_2 = train_data_2_concrete
categorized_labels = categorized_labels_concrete

def reduce_data():
    global train_data_1
    global train_data_2
    global raw_labels
    global categorized_labels
    
    #get number of words in each category
    print("Reduce Data process starting")
    category_amounts = [0 for i in range(11)]
    for i in range(len(raw_labels)):
        category_amounts[int(raw_labels[i])] += 1
    for i in range(len(category_amounts)):
        print(category_amounts[i])
    
    #go through each sample and remove based on fraction
    ceiling = 50000
    print("Marking for deletion starting")
    for i in range(len(raw_labels)):
        random_num = random.random()
        fraction = 1/(category_amounts[int(raw_labels[i])] / ceiling)
        if random_num >= fraction:
            # delete it
            train_data_1[i] = ""
            train_data_2[i] = ""
            categorized_labels[i] = [0.]
    print("Marking for deletion finished")
    print("Making filtered list starting")
    train_data_1 = [t for t in train_data_1 if t != ""]
    train_data_2 = [t for t in train_data_2 if t != ""]
    categorized_labels = [t for t in categorized_labels if sum(t) != 0.]
    print("Making filtered list finished")
    print("Reduce Data process finished")

reduce_data()
category_amounts = [0 for i in range(11)]
for i in range(len(categorized_labels)):
    current_label = categorized_labels[i]
    index = 0
    for u in range(len(current_label)):
        if current_label[u] == 1:
            index = u
    category_amounts[index] += 1
for i in range(len(category_amounts)):
    print(category_amounts[i])

Reduce Data process starting


In [ ]:
# build whole-sequence model
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Subtract, Conv1D

batch_size = 100
output_dim_size = 20

input_1 = Input(batch_shape=(batch_size, 6))
input_2 = Input(batch_shape=(batch_size, 6))
embedding = Embedding(vocabulary_size, output_dim_size, input_length=max_len)
conv1d_1 = Conv1D(100, 6, activation = "relu", strides=1, padding="valid")
conv1d_2 = Conv1D(50, 6, activation = "relu", strides=1, padding="valid")
conv1d_3 = Conv1D(25, 6, activation = "relu", strides=1, padding="valid")

embedding_1 = embedding(input_1)
embedding_2 = embedding(input_2)
conv1d_1_1 = conv1d_1(embedding_1)
conv1d_1_2 = conv1d_1(embedding_2)
conv1d_2_1 = conv1d_2(conv1d_1_1)
conv1d_2_2 = conv1d_2(conv1d_1_2)
conv1d_3_1 = conv1d_3(conv1d_2_1)
conv1d_3_2 = conv1d_3(conv1d_2_2)
merge_layer = Subtract()([conv1d_3_1, conv1d_3_2])
flatten = Flatten()(merge_layer)
dense_output_layer = Dense(11, activation="softmax", input_shape=(max_len*output_dim_size,))(flatten)

whole_sequence_model = Model([input_1, input_2], dense_output_layer)
whole_sequence_model.summary()

In [26]:
# Compile and fit model
print(len(train_data_1))
whole_sequence_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
the_model = whole_sequence_model.fit([np.array(train_data_1[:372500]), np.array(train_data_2[:372500])], 
                         np.array(categorized_labels[:372500]), 
                         epochs=10, batch_size=100, validation_split = 0.2)

372517
Train on 298000 samples, validate on 74500 samples
Epoch 1/10
298000/298000 [==============================] - 21s 72us/step - loss: 2.1387 - acc: 0.1746 - val_loss: 2.0624 - val_acc: 0.1997
Epoch 2/10
298000/298000 [==============================] - 21s 69us/step - loss: 2.0471 - acc: 0.2061 - val_loss: 2.0401 - val_acc: 0.2179
Epoch 3/10
298000/298000 [==============================] - 21s 69us/step - loss: 2.0284 - acc: 0.2198 - val_loss: 2.0275 - val_acc: 0.2185
Epoch 4/10
298000/298000 [==============================] - 21s 69us/step - loss: 2.0172 - acc: 0.2261 - val_loss: 2.0164 - val_acc: 0.2234
Epoch 5/10
298000/298000 [==============================] - 21s 69us/step - loss: 2.0089 - acc: 0.2305 - val_loss: 2.0126 - val_acc: 0.2266
Epoch 6/10
298000/298000 [==============================] - 21s 69us/step - loss: 2.0026 - acc: 0.2324 - val_loss: 2.0083 - val_acc: 0.2303
Epoch 7/10
298000/298000 [==============================] - 21s 69us/step - loss: 1.9977 - acc: 0.2353